# NOMAD Samples Dashboard

This dashboard provides a modern interface for managing and analyzing HySprint samples in NOMAD. Key features:

1. **Authentication**: Secure connection to NOMAD Oasis instances
2. **Sample Management**: View and manage HySprint sample data
3. **Author Attribution**: Track and override sample attributions
4. **Analytics**: Visualize sample statistics and trends

## Setup and Dependencies

In [1]:
# Import required libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Text, Password, Label, Dropdown
from IPython.display import display, clear_output, FileLink
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime, timedelta
import asyncio
from IPython.lib.backgroundjobs import BackgroundJobManager
jobs = BackgroundJobManager()

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Import NOMAD authentication and data modules
%run 'nomad_auth.ipynb'
%run 'nomad_data_retrieval.ipynb'
from nomad_data import load_attributions, save_attributions

## Authentication Tab

In [2]:
def create_auth_tab():
    """Create the authentication tab using nomad_auth functionality"""
    auth_ui = widgets.VBox([
        widgets.HTML("<h3>NOMAD Authentication</h3>"),
        widgets.HTML("<p>Please authenticate with NOMAD to access data.</p>"),
        general_settings_box  # From nomad_auth.ipynb
    ])
    
    # Create wrapper for authentication state
    auth_state = {
        'is_authenticated': lambda: api_client is not None,
        'token': lambda: current_token,
        'user_info': lambda: current_user_info,
        'client': lambda: api_client,
        'oasis': lambda: oasis_dropdown.value if oasis_dropdown.value else None,
        'oasis_url': lambda: oasis_options.get(oasis_dropdown.value, None) if oasis_dropdown.value else None
    }
    
    return auth_ui, auth_state

## Data Retrieval Tab

In [3]:
def initialize_data_tab(auth_state):
    """Create the data retrieval tab using the modular nomad_data_retrieval functionality"""
    # Use the modular data retrieval component from nomad_data_retrieval.ipynb
    return create_data_tab(auth_state)

## Attribution Management Tab

In [4]:
def create_attribution_tab(data_state):
    """Create the attribution management tab"""
    # Save button
    save_button = widgets.Button(
        description='Save Attributions',
        disabled=False,
        button_style='success',
        tooltip='Save attribution changes to local file',
        icon='save'
    )
    
    # Reset button
    reset_button = widgets.Button(
        description='Reset Changes',
        disabled=False,
        button_style='danger',
        tooltip='Reset all attribution changes',
        icon='refresh'
    )
    
    # Status output
    status_output = widgets.Output()
    
    # Create a helper function to get unique authors from the data
    def get_unique_authors():
        # Get unique authors from attributions (override authors)
        override_authors = set()
        for attr in data_state['attributions'].values():
            # Use new field names with fallbacks for backward compatibility
            author = attr.get('author_display_name', 
                     attr.get('main_author_name',
                     attr.get('author_id',
                     attr.get('main_author', ''))))
            if author:
                override_authors.add(author)
        
        # Get unique authors from the dataframe (all authors with uploads)
        upload_authors = set()
        df = data_state['df']
        if df is not None and not df.empty:
            author_col = 'author_name' if 'author_name' in df.columns else 'main_author'
            if author_col in df.columns:
                upload_authors = set(df[author_col].dropna().unique())
        
        # Combine and sort all authors
        all_authors = sorted(list(override_authors.union(upload_authors)))
        return all_authors
    
    # Helper function to get matching uploads based on criteria
    def get_matching_uploads(name_pattern='', date_from='', date_to='', author_pattern=''):
        if data_state['df'] is None or data_state['df'].empty:
            return pd.DataFrame()
        
        # Get a copy of the dataframe with unique upload_ids
        df = data_state['df'].drop_duplicates(subset=['upload_id']).copy()
        filtered_df = df.copy()
        
        # Apply name pattern filter
        if name_pattern:
            filtered_df = filtered_df[filtered_df['upload_name'].str.lower().str.contains(name_pattern.lower(), na=False)]
        
        # Apply date range filter
        if date_from:
            try:
                from_date = pd.to_datetime(date_from)
                filtered_df = filtered_df[pd.to_datetime(filtered_df['upload_date']) >= from_date]
            except:
                pass  # Ignore invalid date format
        
        if date_to:
            try:
                to_date = pd.to_datetime(date_to)
                filtered_df = filtered_df[pd.to_datetime(filtered_df['upload_date']) <= to_date]
            except:
                pass  # Ignore invalid date format
        
        # Apply author pattern filter
        if author_pattern:
            # Check both author_name and main_author columns
            author_col = 'author_name' if 'author_name' in filtered_df.columns else 'main_author'
            filtered_df = filtered_df[filtered_df[author_col].str.lower().str.contains(author_pattern.lower(), na=False)]
        
        return filtered_df
    
    # Create the bulk attribution section
    bulk_container = widgets.VBox()
    bulk_title = widgets.HTML("<h3>Bulk Attribution Editing</h3>")
    bulk_description = widgets.HTML(
        """<p>Use this feature to assign the same author to multiple uploads that match specific criteria.</p>"""
    )
    
    # Create selection criteria inputs
    criteria_container = widgets.VBox()
    
    # Upload name pattern
    name_pattern = widgets.Text(
        placeholder='Enter pattern to match upload names...',
        description='Upload name contains:',
        layout=widgets.Layout(width='350px')
    )
    
    # Author filter
    author_filter = widgets.Text(
        placeholder='Filter by current author...',
        description='Current author contains:',
        layout=widgets.Layout(width='350px')
    )
    
    # Date range filter
    date_filter_row = widgets.HBox()
    date_from = widgets.Text(
        placeholder='YYYY-MM-DD',
        description='From date:',
        layout=widgets.Layout(width='200px')
    )
    date_to = widgets.Text(
        placeholder='YYYY-MM-DD',
        description='To date:',
        layout=widgets.Layout(width='200px')
    )
    date_filter_row.children = [date_from, date_to]
    
    # New author input
    new_author_input = widgets.Combobox(
        options=[],  # Will be populated later
        placeholder='Type or select new author name',
        description='New author:',
        ensure_option=False,  # Allow custom values
        layout=widgets.Layout(width='350px')
    )
    
    # Preview button
    preview_button = widgets.Button(
        description='Preview Matches',
        button_style='info',
        icon='search',
        layout=widgets.Layout(width='150px')
    )
    
    # Apply button
    apply_button = widgets.Button(
        description='Apply to All Matches',
        button_style='warning',
        icon='check',
        disabled=True,
        layout=widgets.Layout(width='180px')
    )
    
    # Results area
    bulk_results = widgets.Output(layout=widgets.Layout(max_height='300px', overflow='auto'))
    
    # Function to update the author options in the dropdown
    def update_author_options():
        authors = get_unique_authors()
        new_author_input.options = authors
    
    # Preview button handler
    def on_preview_button_click(b):
        with bulk_results:
            clear_output()
            
            # Get matches based on criteria
            matches = get_matching_uploads(
                name_pattern=name_pattern.value,
                date_from=date_from.value,
                date_to=date_to.value,
                author_pattern=author_filter.value
            )
            
            if len(matches) == 0:
                print("No uploads match your criteria.")
                apply_button.disabled = True
            else:
                print(f"Found {len(matches)} matching uploads:")
                
                # Create a preview table with limited columns
                preview_df = matches[['upload_id', 'upload_name']].copy()
                
                # Add current author column
                author_col = 'author_name' if 'author_name' in matches.columns else 'main_author'
                preview_df['current_author'] = matches[author_col]
                
                # Show the preview
                display(preview_df)
                
                # Enable apply button if we have matches and a new author
                apply_button.disabled = not (len(matches) > 0 and new_author_input.value)
    
    # Apply button handler
    def on_apply_button_click(b):
        with bulk_results:
            clear_output()
            
            new_author = new_author_input.value
            if not new_author:
                print("⚠️ Please enter a new author name first.")
                return
            
            # Get matches
            matches = get_matching_uploads(
                name_pattern=name_pattern.value,
                date_from=date_from.value,
                date_to=date_to.value,
                author_pattern=author_filter.value
            )
            
            if len(matches) == 0:
                print("No uploads match your criteria.")
                return
            
            # Confirm with the user
            print(f"Applying author '{new_author}' to {len(matches)} uploads...")
            
            # Apply the new author to all matches
            count = 0
            for upload_id in matches['upload_id']:
                data_state['attributions'][upload_id] = {
                    'author_id': new_author,
                    'author_display_name': new_author,
                    'override_date': datetime.now().strftime('%Y-%m-%d')
                }
                count += 1
            
            print(f"✅ Successfully applied author '{new_author}' to {count} uploads.")
            
            # Refresh the main table
            update_table()
            
            # Update author options to include this one if it's new
            update_author_options()
    
    # Connect handlers
    preview_button.on_click(on_preview_button_click)
    apply_button.on_click(on_apply_button_click)
    
    # When author input changes, check if we can enable apply button
    def on_author_change(change):
        # Only enable apply if we have a non-empty value and preview was already shown
        if change['new'] and not apply_button.disabled:
            apply_button.disabled = False
    
    new_author_input.observe(on_author_change, names='value')
    
    # Assemble all components
    criteria_container.children = [
        widgets.HTML("<h4>Selection Criteria</h4>"),
        name_pattern,
        author_filter,
        date_filter_row,
        widgets.HTML("<h4>New Author Assignment</h4>"),
        new_author_input,
        widgets.HBox([preview_button, apply_button]),
        bulk_results
    ]
    
    # Add to bulk container
    bulk_container.children = [
        bulk_title,
        bulk_description,
        criteria_container
    ]
    
    # Style the bulk container
    bulk_container.layout = widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='10px 0',
        border_radius='5px'
    )
    
    # Sample table container
    table_container = widgets.Output()
    
    # Update table function
    def update_table():
        with table_container:
            clear_output()
            if data_state['df'] is not None and not data_state['df'].empty:
                table_widget = create_sample_table(data_state['df'], data_state['attributions'])
                display(table_widget)
            else:
                display(widgets.HTML("<p>No data available. Please fetch data first.</p>"))
    
    # Save button click handler
    def on_save_button_click(b):
        with status_output:
            clear_output()
            if data_state['attributions']:
                success = save_attributions(data_state['attributions'])
                if success:
                    print(f"✓ Saved {len(data_state['attributions'])} attribution overrides")
                else:
                    print("❌ Failed to save attributions")
            else:
                print("ℹ️ No attributions to save")
    
    # Reset button click handler
    def on_reset_button_click(b):
        with status_output:
            clear_output()
            data_state['attributions'] = load_attributions()
            print("✓ Attributions reset to saved state")
            update_table()
    
    # Connect event handlers
    save_button.on_click(on_save_button_click)
    reset_button.on_click(on_reset_button_click)
    
    # Refresh button
    refresh_button = widgets.Button(
        description='Refresh Table',
        button_style='info',
        icon='sync'
    )
    
    def on_refresh_click(b):
        update_table()
        update_author_options()
        
    refresh_button.on_click(on_refresh_click)
    
    # Combine widgets into a form
    attribution_ui = widgets.VBox([
        widgets.HTML("<h2>Sample Attribution Management</h2>"),
        widgets.HBox([refresh_button, save_button, reset_button]),
        status_output,
        bulk_container,
        table_container
    ])
    
    # Initialize table and author options
    update_table()
    update_author_options()
    
    return attribution_ui

## Visualization Tab

In [5]:
def create_visualization_tab(data_state):
    """Create the visualization tab"""
    # Visualization container
    viz_container = widgets.Output()
    
    # Refresh button
    refresh_button = widgets.Button(
        description='Refresh Visualizations',
        button_style='info',
        icon='sync'
    )
    
    # Status output
    status_output = widgets.Output()
    
    # Update visualizations function
    def update_visualizations():
        with viz_container:
            clear_output()
            if data_state['df'] is not None and not data_state['df'].empty:
                df = data_state['df']
                
                # Create author distribution plot using author names instead of IDs
                plt.figure(figsize=(10, 6))
                # Use author_name column if available, otherwise fall back to main_author
                if 'author_name' in df.columns:
                    author_counts = df['author_name'].value_counts()
                else:
                    author_counts = df['main_author'].value_counts()
                # Limit to top 15 authors if there are many
                if len(author_counts) > 15:
                    author_counts = author_counts.head(15)
                    plt.title('Top 15 Authors by Sample Count')
                else:
                    plt.title('Sample Distribution by Author')
                
                # Create the bar plot with author names
                sns.barplot(x=author_counts.values, y=author_counts.index)
                plt.xlabel('Number of Samples')
                plt.tight_layout()
                plt.show()
                
                # Create time series plot
                plt.figure(figsize=(12, 6))
                df['upload_date'] = pd.to_datetime(df['upload_date'])
                samples_by_date = df.groupby('upload_date').size()
                samples_by_date.plot(kind='line', marker='o')
                plt.title('Samples Over Time')
                plt.xlabel('Date')
                plt.ylabel('Number of Samples')
                plt.grid(True)
                plt.tight_layout()
                plt.show()
                
                # Create efficiency distribution plot
                if 'efficiency' in df.columns:
                    plt.figure(figsize=(8, 6))
                    sns.histplot(data=df, x='efficiency', bins=20)
                    plt.title('Distribution of Sample Efficiencies')
                    plt.xlabel('Efficiency (%)')
                    plt.ylabel('Count')
                    plt.tight_layout()
                    plt.show()
                    
                    # Add a boxplot of efficiencies by author
                    if 'author_name' in df.columns and len(df['author_name'].unique()) <= 10:
                        plt.figure(figsize=(12, 7))
                        sns.boxplot(data=df, x='author_name', y='efficiency')
                        plt.title('Sample Efficiency by Author')
                        plt.xlabel('Author')
                        plt.ylabel('Efficiency (%)')
                        plt.xticks(rotation=45, ha='right')
                        plt.tight_layout()
                        plt.show()
            else:
                display(widgets.HTML("<p>No data available. Please fetch data first.</p>"))
    
    # Refresh button click handler
    def on_refresh_click(b):
        with status_output:
            clear_output()
            print("Refreshing visualizations...")
            update_visualizations()
            print("✓ Visualizations updated")
    
    refresh_button.on_click(on_refresh_click)
    
    # Combine widgets into a form
    viz_ui = widgets.VBox([
        widgets.HTML("<h2>Sample Visualizations</h2>"),
        refresh_button,
        status_output,
        viz_container
    ])
    
    # Initialize visualizations
    update_visualizations()
    
    return viz_ui

## Main Dashboard

In [6]:
def create_sample_table(df, attributions):
    """Create an interactive table for sample attribution management with pagination and filtering"""
    # State variables for pagination and filtering
    pagination_state = {
        'current_page': 0,
        'items_per_page': 20,
        'filtered_df': None
    }
    
    filter_state = {
        'author_filter': '',
        'upload_date_from': '',
        'upload_date_to': '',
        'upload_name_filter': '',
        'upload_id_filter': ''
    }
    
    # Main container for the entire table
    main_container = widgets.VBox()
    
    # Add a persistent notification at the top of the table
    save_reminder = widgets.HTML(
        """<div style="background-color: #fff3cd; color: #856404; padding: 8px; 
        border-left: 5px solid #ffeeba; margin-bottom: 10px; border-radius: 3px;">
        <b>⚠️ Important:</b> Click individual row "Save" buttons to save changes temporarily. 
        Click the <b>"Save Attributions"</b> button at the top to make all changes permanent.
        </div>"""
    )
    
    # Create a subset of the dataframe with unique upload_ids
    # This ensures we don't show duplicate entries in the attribution table
    unique_df = df.drop_duplicates(subset=['upload_id']).copy()
    
    # Add a note about deduplication if needed
    dedup_note = None
    if len(unique_df) < len(df):
        dedup_note = widgets.HTML(
            f"""<div style="background-color: #d1ecf1; color: #0c5460; padding: 8px; 
            border-left: 5px solid #bee5eb; margin-bottom: 10px; border-radius: 3px;">
            Note: Showing {len(unique_df)} unique uploads out of {len(df)} total entries.
            </div>"""
        )
    
    # Status message area for individual row actions
    row_status = widgets.HTML("", layout=widgets.Layout(margin='5px 0'))
    
    # Prepare author dropdown options
    def get_author_options():
        # Get unique authors from attributions (authors used for overrides)
        override_authors = set()
        for attr in attributions.values():
            # Use new field names with fallbacks for backward compatibility
            author = attr.get('author_display_name', 
                     attr.get('main_author_name',
                     attr.get('author_id',
                     attr.get('main_author', ''))))
            if author:
                override_authors.add(author)
        
        # Get unique authors from the dataframe (all authors with uploads)
        upload_authors = set()
        author_col = 'author_name' if 'author_name' in df.columns else 'main_author'
        if author_col in df.columns:
            upload_authors = set(df[author_col].dropna().unique())
        
        # Create sorted lists
        sorted_override_authors = sorted(list(override_authors))
        # Only include upload authors that aren't already in override_authors
        remaining_authors = sorted(list(upload_authors - override_authors))
        
        # Combine lists with a separator
        if sorted_override_authors and remaining_authors:
            return sorted_override_authors + ['─────────────'] + remaining_authors
        elif sorted_override_authors:
            return sorted_override_authors
        else:
            return remaining_authors
    
    # Get author options once for all dropdowns
    author_options = get_author_options()
    
    # Function to apply filters to the dataframe
    def apply_filters(df):
        filtered = df.copy()
        
        # Filter by author name
        if filter_state['author_filter']:
            author_filter = filter_state['author_filter'].lower()
            # Check both author_name and main_author columns
            author_mask = filtered['author_name'].str.lower().str.contains(author_filter, na=False)
            if 'main_author' in filtered.columns:
                author_mask = author_mask | filtered['main_author'].str.lower().str.contains(author_filter, na=False)
            filtered = filtered[author_mask]
        
        # Filter by upload date range
        if filter_state['upload_date_from']:
            try:
                from_date = pd.to_datetime(filter_state['upload_date_from'])
                filtered = filtered[pd.to_datetime(filtered['upload_date']) >= from_date]
            except:
                pass  # Ignore invalid date format
                
        if filter_state['upload_date_to']:
            try:
                to_date = pd.to_datetime(filter_state['upload_date_to'])
                filtered = filtered[pd.to_datetime(filtered['upload_date']) <= to_date]
            except:
                pass  # Ignore invalid date format
        
        # Filter by upload name
        if filter_state['upload_name_filter']:
            name_filter = filter_state['upload_name_filter'].lower()
            filtered = filtered[filtered['upload_name'].str.lower().str.contains(name_filter, na=False)]
        
        # Filter by upload ID
        if filter_state['upload_id_filter']:
            id_filter = filter_state['upload_id_filter'].lower()
            filtered = filtered[filtered['upload_id'].str.lower().str.contains(id_filter, na=False)]
        
        return filtered
    
    # Create filter UI
    filter_container = widgets.VBox()
    
    filter_title = widgets.HTML("<h4>Filter Uploads</h4>")
    
    # Author filter
    author_filter = widgets.Text(
        placeholder='Filter by author name...',
        description='Author:',
        layout=widgets.Layout(width='300px')
    )
    
    # Upload date range filter
    date_filter_row = widgets.HBox()
    upload_date_from = widgets.Text(
        placeholder='YYYY-MM-DD',
        description='From date:',
        layout=widgets.Layout(width='200px')
    )
    upload_date_to = widgets.Text(
        placeholder='YYYY-MM-DD',
        description='To date:',
        layout=widgets.Layout(width='200px')
    )
    date_filter_row.children = [upload_date_from, upload_date_to]
    
    # Upload name filter
    upload_name_filter = widgets.Text(
        placeholder='Filter by upload name...',
        description='Upload name:',
        layout=widgets.Layout(width='300px')
    )
    
    # Upload ID filter
    upload_id_filter = widgets.Text(
        placeholder='Filter by upload ID...',
        description='Upload ID:',
        layout=widgets.Layout(width='300px')
    )
    
    # Apply filter button
    apply_filter_button = widgets.Button(
        description='Apply Filters',
        button_style='info',
        icon='filter'
    )
    
    # Clear filter button
    clear_filter_button = widgets.Button(
        description='Clear Filters',
        button_style='warning',
        icon='eraser'
    )
    
    filter_buttons = widgets.HBox([apply_filter_button, clear_filter_button])
    
    filter_container.children = [
        filter_title,
        widgets.HBox([author_filter, upload_name_filter]), 
        date_filter_row,
        upload_id_filter,
        filter_buttons
    ]
    
    # Table container for the sample data
    table_container = widgets.VBox()
    
    # Create pagination UI
    pagination_container = widgets.HBox(layout=widgets.Layout(margin='10px 0', 
                                                        justify_content='space-between',
                                                        width='100%'))
    
    prev_button = widgets.Button(
        description='Previous',
        button_style='info',
        disabled=True,
        icon='arrow-left'
    )
    
    next_button = widgets.Button(
        description='Next',
        button_style='info',
        disabled=True,
        icon='arrow-right'
    )
    
    page_indicator = widgets.HTML("Page 1")
    
    pagination_container.children = [prev_button, page_indicator, next_button]
    
    # Function to create table rows based on current pagination and filters
    def create_table_rows():
        # Get filtered dataframe
        if pagination_state['filtered_df'] is None:
            pagination_state['filtered_df'] = apply_filters(unique_df)
        
        filtered_df = pagination_state['filtered_df']
        
        # Calculate pagination
        start_idx = pagination_state['current_page'] * pagination_state['items_per_page']
        end_idx = start_idx + pagination_state['items_per_page']
        
        # Get page data
        page_df = filtered_df.iloc[start_idx:end_idx]
        
        # Update pagination controls
        total_pages = max(1, (len(filtered_df) + pagination_state['items_per_page'] - 1) // pagination_state['items_per_page'])
        current_page_num = pagination_state['current_page'] + 1  # 1-based for display
        
        prev_button.disabled = current_page_num <= 1
        next_button.disabled = current_page_num >= total_pages
        
        page_indicator.value = f"Page {current_page_num} of {total_pages} (showing {len(page_df)} of {len(filtered_df)} entries)"
        
        # Create header row
        rows = []
        header = widgets.HBox([
            widgets.Label('Upload ID', layout=widgets.Layout(width='250px')),
            widgets.Label('Upload Name', layout=widgets.Layout(width='200px')),
            widgets.Label('Current Author', layout=widgets.Layout(width='200px')),
            widgets.Label('Override Author', layout=widgets.Layout(width='200px')),
            widgets.Label('Actions', layout=widgets.Layout(width='100px'))
        ], layout=widgets.Layout(margin='5px 0', font_weight='bold'))
        rows.append(header)
        
        # Create rows for each sample on the current page
        for _, row in page_df.iterrows():
            upload_id = row['upload_id']
            # Use author_name instead of main_author if available
            current_author = row.get('author_name', row.get('main_author', 'Unknown'))
            
            # Get the current override value if it exists
            override_value = ''
            if upload_id in attributions:
                # Use the new field names with fallbacks for backward compatibility
                override_value = attributions[upload_id].get('author_display_name', 
                                 attributions[upload_id].get('main_author_name',
                                 attributions[upload_id].get('author_id',
                                 attributions[upload_id].get('main_author', ''))))
            
            # Create dropdown for override author with proper error handling
            # Prepare dropdown options - simple list for Combobox
            dropdown_options = []
            
            # Add authors used for overrides first
            override_authors = []
            regular_authors = []
            
            for author in author_options:
                # Skip separator marker from display options
                if author == '─────────────':
                    continue
                    
                # Used attributions will be at the top of the list
                if any(attr.get('author_display_name') == author or 
                       attr.get('main_author_name') == author or 
                       attr.get('author_id') == author or 
                       attr.get('main_author') == author 
                       for attr in attributions.values()):
                    override_authors.append(author)
                else:
                    regular_authors.append(author)
            
            # Sort each section alphabetically
            override_authors.sort()
            regular_authors.sort()
            
            # Create the final options list
            dropdown_options = override_authors
            if override_authors and regular_authors:
                dropdown_options.append('─────────────')  # Add separator
            dropdown_options.extend(regular_authors)
            
            # Create the combobox widget with proper layout settings
            override_input = widgets.Combobox(
                options=dropdown_options,
                value=override_value,
                placeholder='Type or select author',
                ensure_option=False,  # Allow custom values not in the dropdown
                layout=widgets.Layout(
                    width='200px',
                    margin='0px',
                    padding='0px'
                ),
                description=''  # No label
            )
            
            # Create save button
            save_button = widgets.Button(
                description='Save',
                button_style='primary',
                layout=widgets.Layout(width='80px')
            )
            
            # Define save button click handler
            def make_save_handler(btn_upload_id, btn_dropdown, btn_save):
                def save_handler(b):
                    # Get input value and strip all whitespace and non-visible characters
                    raw_value = btn_dropdown.value
                    override_value = "".join(c for c in raw_value if c.isprintable() and not c.isspace())
                    
                    # Skip separator line if selected
                    if override_value == "─────────────":
                        return
                    
                    # If the cleaned value is different from the original, update the dropdown
                    if raw_value != override_value and override_value:
                        btn_dropdown.value = override_value
                    
                    # Provide immediate visual feedback
                    btn_save.description = 'Saved!'
                    btn_save.button_style = 'success'  # Change to green
                    btn_save.icon = 'check'
                    
                    if override_value:
                        # Store using the new field names
                        attributions[btn_upload_id] = {
                            'author_id': override_value,
                            'author_display_name': override_value,  # Initially the same as what user entered
                            'override_date': datetime.now().strftime('%Y-%m-%d')
                        }
                        btn_dropdown.style.background = '#d4f7d4'  # Light green
                        row_status.value = f"<span style='color: green'>✓ Attribution saved for upload {btn_upload_id}</span>"
                    else:
                        if btn_upload_id in attributions:
                            del attributions[btn_upload_id]
                        btn_dropdown.style.background = ''
                        row_status.value = f"<span style='color: blue'>ℹ Attribution removed for upload {btn_upload_id}</span>"
                    
                    # Schedule the button to revert after a delay
                    async def reset_after_delay():
                        await asyncio.sleep(1.5)  # Wait for 1.5 seconds
                        btn_save.description = 'Save'
                        btn_save.button_style = 'primary'  # Change back to original style
                        btn_save.icon = ''
                        # Clear status message after a delay
                        await asyncio.sleep(1.5)  # Additional delay before clearing the message
                        row_status.value = ""
                    
                    # Create and run the asyncio task
                    try:
                        loop = asyncio.get_event_loop()
                    except RuntimeError:
                        loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(loop)
                    
                    # Run the async task in the current event loop
                    if loop.is_running():
                        asyncio.ensure_future(reset_after_delay())  # For running event loop
                    else:
                        loop.run_until_complete(reset_after_delay())  # For non-running loop
                return save_handler
            
            save_button.on_click(make_save_handler(upload_id, override_input, save_button))
            
            # Highlight overridden values - check for either new or old field names
            if upload_id in attributions and (
                attributions[upload_id].get('author_id') or 
                attributions[upload_id].get('author_display_name') or
                attributions[upload_id].get('main_author')):
                override_input.style.background = '#d4f7d4'
            
            # Create row
            row_widget = widgets.HBox([
                widgets.Label(upload_id, layout=widgets.Layout(width='250px')),
                widgets.Label(row.get('upload_name', 'Unknown'), layout=widgets.Layout(width='200px')),
                widgets.Label(current_author, layout=widgets.Layout(width='200px')),
                override_input,
                save_button
            ], layout=widgets.Layout(margin='2px 0'))
            
            rows.append(row_widget)
            
        return rows
    
    # Function to update the table with current pagination and filters
    def update_table():
        # Apply filters and update filtered dataframe
        pagination_state['filtered_df'] = apply_filters(unique_df)
        
        # Reset to first page when filters change
        pagination_state['current_page'] = 0
        
        # Create table rows
        rows = create_table_rows()
        
        # Update table container
        table_container.children = rows
    
    # Function handlers for pagination
    def on_prev_button_click(b):
        if pagination_state['current_page'] > 0:
            pagination_state['current_page'] -= 1
            table_container.children = create_table_rows()
    
    def on_next_button_click(b):
        filtered_df = pagination_state['filtered_df']
        total_pages = (len(filtered_df) + pagination_state['items_per_page'] - 1) // pagination_state['items_per_page']
        
        if pagination_state['current_page'] < total_pages - 1:
            pagination_state['current_page'] += 1
            table_container.children = create_table_rows()
    
    # Function handlers for filters
    def on_apply_filter_click(b):
        # Update filter state
        filter_state['author_filter'] = author_filter.value
        filter_state['upload_date_from'] = upload_date_from.value
        filter_state['upload_date_to'] = upload_date_to.value
        filter_state['upload_name_filter'] = upload_name_filter.value
        filter_state['upload_id_filter'] = upload_id_filter.value
        
        # Update the table
        update_table()
    
    def on_clear_filter_click(b):
        # Clear filter inputs
        author_filter.value = ''
        upload_date_from.value = ''
        upload_date_to.value = ''
        upload_name_filter.value = ''
        upload_id_filter.value = ''
        
        # Clear filter state
        filter_state['author_filter'] = ''
        filter_state['upload_date_from'] = ''
        filter_state['upload_date_to'] = ''
        filter_state['upload_name_filter'] = ''
        filter_state['upload_id_filter'] = ''
        
        # Update the table
        update_table()
    
    # Connect handlers
    prev_button.on_click(on_prev_button_click)
    next_button.on_click(on_next_button_click)
    apply_filter_button.on_click(on_apply_filter_click)
    clear_filter_button.on_click(on_clear_filter_click)
    
    # Initialize filtered dataframe
    pagination_state['filtered_df'] = unique_df
    
    # Build the main container structure
    main_container_children = [save_reminder]
    
    if dedup_note:
        main_container_children.append(dedup_note)
    
    main_container_children.extend([
        filter_container,
        row_status,
        table_container,
        pagination_container
    ])
    
    main_container.children = main_container_children
    
    # Initialize the table
    table_container.children = create_table_rows()
    
    return main_container

def create_dashboard():
    """Create and display the main dashboard"""
    # Create authentication tab
    auth_tab, auth_state = create_auth_tab()
    
    # Create data tab using the modular component
    data_tab, data_state = initialize_data_tab(auth_state)
    
    # Create attribution tab
    attribution_tab = create_attribution_tab(data_state)
    
    # Create visualization tab
    viz_tab = create_visualization_tab(data_state)
    
    # Create tab widget
    tab = widgets.Tab([
        auth_tab,
        data_tab,
        attribution_tab,
        viz_tab
    ])
    
    tab.set_title(0, 'Authentication')
    tab.set_title(1, 'Data Retrieval')
    tab.set_title(2, 'Attribution Management')
    tab.set_title(3, 'Visualizations')
    
    # Create dashboard container with header
    dashboard = widgets.VBox([
        widgets.HTML("""
        <div style="background-color: #4CAF50; color: white; padding: 10px; text-align: center; border-radius: 5px;">
            <h1>NOMAD Samples Dashboard</h1>
            <p>Manage and analyze HySprint samples in NOMAD</p>
        </div>
        """),
        tab
    ])
    
    return dashboard

# Create and display the dashboard
dashboard = create_dashboard()
display(dashboard)

Loaded 1 attribution overrides
